In [1]:
import pandas as pd
import datetime as dt
import time
import sys
from pybit.unified_trading import HTTP


In [22]:
############################################
# Add these parameters before running code #
############################################

CATEGORY = "linear"  # linear are perpetual swaps/futures
SYMBOL = "SOLUSDT"
INTERVAL = "D"  # 1,3,5,15,30,60,120,240,360,720,D,W,M
START = int(dt.datetime(2021, 10, 1, 0, 0).timestamp()) * 1000
END = int(dt.datetime(2025, 3, 18, 0, 0).timestamp()) * 1000
SLEEP_TIME = 3  # seconds to sleep between requests
CSV_FILE = f"/opt/bybit/data/{SYMBOL}_{INTERVAL}_20211001.csv"

CHUNK_SIZE_MAPPING = {"1":60,"3":180,"5":300,"15":900,"30":1800,"60":3600,"120":7200,"240":14400,"360":7200*3,"720":7200*6,"D":3600*24}
CHUNK_SIZE = 1000 * CHUNK_SIZE_MAPPING.get(INTERVAL)  # time of chunk

############################################################################################

# For additional info about API, go to https://bybit-exchange.github.io/docs/v5/market/kline
# Initialize session
session = HTTP(testnet=False)


# Helper function to fetch and process data
def fetch_data(start, end):
    response = session.get_kline(
        category=CATEGORY,
        symbol=SYMBOL,
        interval=INTERVAL,
        start=start,
        end=end,
    )

    if response["retMsg"] == "OK" and response["result"]["list"]:
        data = response["result"]["list"]
        columns = ["time", "open", "high", "low", "close", "volume", "turnover"]
        df = pd.DataFrame(data, columns=columns)

        df["time"] = pd.to_numeric(df["time"], errors="coerce")
        df.insert(0, "human", pd.to_datetime(df["time"], unit="ms", utc=True))
        df["human"] = df["human"].dt.tz_convert("Europe/Zagreb")

        return df.iloc[::-1].reset_index(drop=True)

    else:
        print(
            "---------------------------------------------------\n"
            f"start={pd.to_datetime(start, unit='ms', utc=True).tz_convert('Europe/Zagreb')}\n"
            f"end={pd.to_datetime(end, unit='ms', utc=True).tz_convert('Europe/Zagreb')}\n"
            f"response={response}"
        )
        sys.exit(0)



try:
    df_existing = pd.read_csv(CSV_FILE, index_col=False).tail(1)
except FileNotFoundError:
    df_existing = pd.DataFrame(
        columns=["human", "time", "open", "high", "low", "close", "volume", "turnover"]
    )
    df_existing.to_csv(CSV_FILE, index=False)

if df_existing.empty:
    current_start = START
else:
    current_start = int(df_existing["time"].max()) + CHUNK_SIZE

while current_start < END:
    current_end = min((current_start + (CHUNK_SIZE * 200)), END)  # 200 is fetch limit

    df_new = fetch_data(current_start, current_end)
    df_new.to_csv(CSV_FILE, mode="a", index=False, header=False)

    current_start = int(df_new["time"].max()) + CHUNK_SIZE

    print(
        pd.to_datetime(df_new["time"].min(), unit="ms", utc=True).tz_convert(
            "Europe/Zagreb"
        )
    )

    time.sleep(SLEEP_TIME)

2025-03-17 01:00:00+01:00
